In [1]:
#Name: Charlos Pardomuan Purba
#NIM: 11S22054

from collections import deque

In [2]:
#Nama : Charlos Pardomuan Purba
#Nim : 11S22054

# Mendefinisikan ukuran grid lingkungan
GRID_SIZE = 4

# Mendefinisikan lingkungan
class Grid:
    def __init__(self):
        # Inisialisasi grid dengan atribut default
        self.grid = [[{'Pit': False, 'Wumpus': False, 'Gold': False, 'Breeze': False, 'Stench': False}
                      for _ in range(GRID_SIZE + 1)] for _ in range(GRID_SIZE + 1)]
        self.populate_environment()

    def populate_environment(self):
        # Menempatkan Pit
        pits = [(3, 1), (3, 3), (4, 4)]
        for x, y in pits:
            self.grid[x][y]['Pit'] = True

        # Menempatkan Wumpus
        self.grid[1][3]['Wumpus'] = True

        # Menempatkan Emas
        self.grid[2][3]['Gold'] = True

        # Menempatkan Breeze
        breezes = [(2, 1), (4, 1), (3, 2), (4, 3), (3, 4), (2, 3)]
        for x, y in breezes:
            self.grid[x][y]['Breeze'] = True

        # Menempatkan Stench
        stenches = [(1, 2), (1, 4), (2, 3)]
        for x, y in stenches:
            self.grid[x][y]['Stench'] = True

    def get_cell(self, x, y):
        if 1 <= x <= GRID_SIZE and 1 <= y <= GRID_SIZE:
            return self.grid[x][y]
        else:
            return None


In [6]:
#Nama : Charlos Pardomuan Purba
#Nim : 11S22054

# Mendefinisikan Agen
class Agent:
    def __init__(self, environment):
        self.env = environment
        self.x = 1  # Posisi X awal agen
        self.y = 1  # Posisi Y awal agen
        self.visited = set()
        self.safe = set()
        self.frontier = set()
        self.potential_pits = {}    # Menyimpan kemungkinan Pit di sel-sel tertentu
        self.potential_wumpus = {}  # Menyimpan kemungkinan Wumpus di sel-sel tertentu
        self.pit_confirmed = set()  # Menyimpan sel-sel yang dikonfirmasi memiliki Pit
        self.path = []
        self.has_gold = False
        self.kb = {}  # Basis Pengetahuan (Knowledge Base)

    def move(self, x, y):
        self.x = x
        self.y = y
        self.path.append((x, y))
        print(f"Pindah ke ({x}, {y})")

    def perceive(self):
        cell = self.env.get_cell(self.x, self.y)
        percepts = []
        if cell['Breeze']:
            percepts.append('Breeze')
        if cell['Stench']:
            percepts.append('Stench')
        if cell['Gold']:
            percepts.append('Glitter')
            self.has_gold = True
            print(f"Emas ditemukan di ({self.x}, {self.y})!")
        print(f"Persepsi di ({self.x}, {self.y}): {percepts}")
        return percepts

    def update_kb(self, percepts):
        # Menandai sel saat ini sebagai aman
        self.safe.add((self.x, self.y))
        self.visited.add((self.x, self.y))
        adj_cells = self.get_adjacent_cells(self.x, self.y)

        # Penerapan Forward Chaining:
        # Menggunakan persepsi untuk memperbarui basis pengetahuan dan menarik kesimpulan baru.

        # Jika ada Breeze, kemungkinan ada Pit di sel tetangga
        if 'Breeze' in percepts:
            # Penerapan Modus Ponens:
            # Jika ada Breeze di sel saat ini, maka setidaknya satu sel tetangga mengandung Pit.
            # (Breeze(x, y) → ∃ Pit(adjacent(x, y)))
            for cell in adj_cells:
                if cell not in self.safe and cell not in self.visited and cell not in self.pit_confirmed:
                    # Meningkatkan kemungkinan adanya Pit di sel tetangga
                    self.potential_pits[cell] = self.potential_pits.get(cell, 0) + 1
        else:
            # Penerapan Modus Tollens:
            # Jika tidak ada Breeze di sel saat ini, maka tidak ada Pit di sel tetangga.
            # (¬Breeze(x, y) → ¬Pit(adjacent(x, y)))
            for cell in adj_cells:
                if cell in self.potential_pits:
                    del self.potential_pits[cell]  # Menghapus sel dari daftar kemungkinan Pit
                self.safe.add(cell)  # Menandai sel tetangga sebagai aman

        if 'Stench' in percepts:
            # Penerapan Modus Ponens (serupa dengan Breeze, namun untuk Wumpus)
            # Jika ada Stench di sel saat ini, maka setidaknya satu sel tetangga mengandung Wumpus.
            # (Stench(x, y) → ∃ Wumpus(adjacent(x, y)))
            pass  # Implementasi dapat ditambahkan jika diperlukan
        else:
            # Penerapan Modus Tollens (serupa dengan Breeze, namun untuk Wumpus)
            # Jika tidak ada Stench di sel saat ini, maka tidak ada Wumpus di sel tetangga.
            # (¬Stench(x, y) → ¬Wumpus(adjacent(x, y)))
            pass  # Implementasi dapat ditambahkan jika diperlukan

        # Memperbarui frontier dengan sel-sel tetangga yang belum aman atau dikunjungi
        for cell in adj_cells:
            if cell not in self.safe and cell not in self.visited and cell not in self.pit_confirmed:
                self.frontier.add(cell)

        # Mengonfirmasi Pit jika memungkinkan
        self.confirm_pits()

    def get_adjacent_cells(self, x, y):
        adj = []
        if x > 1:
            adj.append((x - 1, y))
        if x < GRID_SIZE:
            adj.append((x + 1, y))
        if y > 1:
            adj.append((x, y - 1))
        if y < GRID_SIZE:
            adj.append((x, y + 1))
        return adj

    def confirm_pits(self):
        # Penerapan Forward Chaining dan Modus Ponens:
        # Mengonfirmasi keberadaan Pit berdasarkan akumulasi bukti
        for cell in list(self.potential_pits):
            if self.potential_pits[cell] >= 2:
                # Jika kemungkinan Pit tinggi (misalnya, lebih dari satu indikasi), konfirmasikan Pit
                # Ini menggunakan inferensi bahwa jika beberapa aturan mendukung kesimpulan yang sama, maka kesimpulan tersebut kuat
                self.pit_confirmed.add(cell)
                self.frontier.discard(cell)
                del self.potential_pits[cell]

    def infer_safe_cells(self):
        # Menginferensi sel aman berdasarkan basis pengetahuan
        # Penerapan Modus Tollens:
        # Jika tidak ada indikasi adanya Pit di sel tetangga, sel tersebut dianggap aman
        for cell in list(self.frontier):
            if cell not in self.potential_pits:
                # Jika sel tidak ada dalam potential_pits, anggap aman
                self.safe.add(cell)
                self.frontier.discard(cell)

    def choose_next_move(self):
        # Memilih gerakan berikutnya berdasarkan sel aman yang belum dikunjungi
        adj_cells = self.get_adjacent_cells(self.x, self.y)
        unvisited_safe = [cell for cell in adj_cells if cell in self.safe and cell not in self.visited]
        if unvisited_safe:
            return unvisited_safe[0]

        # Jika tidak ada sel aman di sekitar, mencari jalur ke sel aman lain
        # Penerapan Forward Chaining:
        # Menggunakan informasi yang ada untuk menemukan jalur ke sel aman
        for cell in self.safe:
            if cell not in self.visited:
                path = self.find_path((self.x, self.y), cell)
                if path:
                    return path[1]  # Pindah ke sel berikutnya dalam jalur

        # Jika tidak ada opsi, agen terjebak
        return None

    def find_path(self, start, goal):
        # BFS untuk menemukan jalur terpendek ke tujuan
        # Penerapan Forward Chaining:
        # Menggunakan penelusuran maju untuk menemukan jalur berdasarkan informasi yang ada
        queue = deque()
        queue.append([start])
        visited = set()
        while queue:
            path = queue.popleft()
            cell = path[-1]
            if cell == goal:
                return path
            if cell in visited:
                continue
            visited.add(cell)
            for neighbor in self.get_adjacent_cells(*cell):
                if neighbor in self.safe and neighbor not in visited:
                    new_path = list(path)
                    new_path.append(neighbor)
                    queue.append(new_path)
        return None

    def run(self):
        while not self.has_gold:
            percepts = self.perceive()
            self.update_kb(percepts)
            self.infer_safe_cells()
            if self.has_gold:
                # Berhenti setelah mendapatkan emas
                break
            next_move = self.choose_next_move()
            if next_move:
                self.move(*next_move)
            else:
                print("Tidak ada gerakan aman yang tersedia. Agen terjebak.")
                break

        print("\nPermainan selesai. Agen telah menemukan emas!")
        print("\nJalur yang dilalui agen:")
        for idx, step in enumerate(self.path):
            print(f"Langkah {idx + 1}: {step}")


In [7]:
#Nama : Charlos Pardomuan Purba
#Nim : 11S22054

# Inisialisasi lingkungan dan agen
env = Grid()
agent = Agent(env)


In [8]:
# Menjalankan agen
agent.run()


Persepsi di (1, 1): []
Pindah ke (2, 1)
Persepsi di (2, 1): ['Breeze']
Pindah ke (1, 1)
Persepsi di (1, 1): []
Pindah ke (1, 2)
Persepsi di (1, 2): ['Stench']
Pindah ke (2, 2)
Persepsi di (2, 2): []
Pindah ke (3, 2)
Persepsi di (3, 2): ['Breeze']
Pindah ke (2, 2)
Persepsi di (2, 2): []
Pindah ke (2, 3)
Emas ditemukan di (2, 3)!
Persepsi di (2, 3): ['Breeze', 'Stench', 'Glitter']

Permainan selesai. Agen telah menemukan emas!

Jalur yang dilalui agen:
Langkah 1: (2, 1)
Langkah 2: (1, 1)
Langkah 3: (1, 2)
Langkah 4: (2, 2)
Langkah 5: (3, 2)
Langkah 6: (2, 2)
Langkah 7: (2, 3)
